<p float="left">
  <img style="float: right;" src="./imgs/logo1.png" width="100" />
  <img style="float: left;" src="./imgs/ipa.png" width="300" /> 
</p>

<img src="./imgs/div.png" width="100%" /> 

 
# 1. Markov Chain Monte Carlo Methode
1.1 Einführung: Modellierung eines <span style="color:#179c80"><b>Münzwurfs</b></span> <br>
1.2 <span style="color:#179c80"><b>Monte Carlo</b></span> Experiment <br>
1.3 Markov Chain Monte Carlo: <span style="color:#179c80"><b>Random Walk</b></span> <br>
1.4 Markov Chain Monte Carlo: <span style="color:#179c80"><b>Metropolis-Hastings</b></span> Algorithmus <br>
1.5 MCMC and <span style="color:#179c80"><b>Bayes'sche Neuronale Netze</b></span>

## 1.1 Einführung: Modellierung eines Münzwurfs
Unser <span style="color:#179c80"><b>Ziel</b></span> ist die Berechnung der Wahrscheinlichkeit des Events <span style="color:#db8642">Kopf</span> eines Münzwurfs, unter Berücksichtigung von Daten. Formell sind wir also an $p(\text{Kopf}|\text{Daten})$ bzw. $p(\theta|y)$ interessiert. Um Daten sammeln gehen wir davon aus, dass wir die Münze <span style="color:#db8642">zehn</span> mal geworfen haben und dabei <span style="color:#db8642">sechs</span> mal <span style="color:#db8642">Zahl</span> und <span style="color:#db8642">vier</span> mal <span style="color:#db8642">Kopf</span> aufgetaucht ist. Als  <span style="color:#179c80"><b>Likelihood</b></span> bietet es sich deshalb ein eine <span style="color:#db8642">Binomial</span> Verteilung zu verwenden:

$$p(y|\theta) =  {\text{Binomial}(n, y)},$$ 
mit $n=10$ und $y=4$. 

<br>

Fehlt noch der <span style="color:#179c80"><b>Pior</b></span>. Einfachheitshalber nehmen wir im folgenden Experiment einen sogenannten <span style="color:#db8642">uninformative</span> Prior an, d.h. wir haben kein konkretes vorheriges Wissen über unsere Zielverteilung. Dies können wir mithilfe der folgenden <span style="color:#db8642">Beta</span> Verteilung ausdrücken:

$$p(\theta)={\text{Beta}(\alpha,\beta)},$$ 
mit $\alpha=\beta=1$.

<br>

Da die Beta Verteilung ein <span style="color:#db8642">conjugate</span>  Prior zur Binomial Verteilung ist, ist die <span style="color:#179c80"><b>Posterior</b></span> Verteilung ebenfalls Beta-verteilt und die entsprechenden Parameter lassen sich analytisch exakt berechnen: 

$$p(\theta|y) \sim \text{Beta}( \alpha + y, n - y + \beta) = \text{Beta}(5, 7)$$

Das bedeuted, dass wir für diesen speziellen Fall kein MCMC-Verfahren benötigen, da die Posterior-Verteilung <span style="color:#db8642">analytisch exakt</span> berechnet werden kann. Zum besseren Verständis von  <span style="color:#179c80"><b>Markov Chain Monte Carlo</b></span> (MCMC) Methoden  versuchen wir dennoch die exakte Posterior mithilfe des <span style="color:#179c80"><b>Metropolis-Hastings</b></span> Algorithmus anzunähern. Zunächst sollte jedoch das Zusammenspiel von Prior, Likelihood und Posterior verstanden werden.

  <div class="alert alert-block alert-info" style="border-color:white">
    <b>Aufgabe</b>: Betrachten Sie folgende Animation über den Einfluss der Prior auf die Posterior Verteilung. Was fällt Ihnen bei einem Prior von 
$p(\theta) =  {\text{Beta}(1,1)}$ auf? Wie verändert sich dies für größere $\alpha$ und $\beta$ Werte? Nehmen Sie dabei Bezug auf die Begriffe <em>uninformative</em> und <em>informative</em>.
</div>
<img src="./imgs/line.gif" width="400" height="400" />

> <b>Antwort</b>: ...

###### <div class="alert alert-block alert-success" style="background-color:#b6dad2;color:#179c80;border-color:white"> <b>Lösung</b> <small style="color:#179c80">(Klicken auf den Pfeil um die Lösung aufzuklappen)</small></span></div>

> Der Prior <em>Beta(1,1)</em> ordnet <span style="color:#db8642">allen</span> Möglichkeiten  den Wert <span style="color:#db8642">eins</span> zu. Setzt man dies in die Bayes' Regel ein, so ist es nur logisch, dass die Likelihood und resultierende Posterior Verteilung die gleiche Form besitzen.  Dies ist ein wichtiges Merkmal des Bayes'schen Grundgedankens: die Posterior-Verteilung ist in der Regel <span style="color:#db8642">äquivalent</span> (oder proportional) zur Likelihood-Funktion, wenn wir völlig <span style="color:#db8642">uninformative</span> Prior Verteilungen verwenden.
<br>

> Für größere Parameterwerte der Beta Verteilung kodiert der Prior zunehmend mehr <span style="color:#db8642">Informationen</span>, weshalb die Posterior stärker vom Prior beeinflusst wird. Besitzen wir also einen <span style="color:#db8642">vorherigen</span> Glauben über die Zielverteilung, so können wir diese Überzeugung mithilfe eines  <span style="color:#db8642">informative</span>  Prior ausdrücken - je stärker dieser Glauben, desto weniger Einfluss haben die Daten auf die Posterior Verteilung .

#####  
<img src="./imgs/div.png" width="100%" /> 

 
## 1.2 Monte Carlo Experiment
Der Metropolis-Hastings Algorithmus ordnet sich in die Reihe der <span style="color:#179c80"><b>Markov Chain Monte Carlo</b> </span> Methoden ein. Bevor wir also mit dessen Implementierung beginnen können, müssen zunächst zwei Begriffe erläutert werden. In diesem Kapitel soll es um <span style="color:#179c80"><b>Monte Carlo</b> </span> Methoden im Allgemeinen gehen.

Bei <span style="color:#179c80"><b>Monte Carlo</b> </span> Verfahren werden Stichproben einer Verteilunge, auch <span style="color:#db8642">Samples</span> genannt, wiederholt <span style="color:#179c80"><b>zufällig</b> </span> gezogen. Die Verteilung, mit der pro Iteration neue Stichproben gezogen werden, wird üblicherweise <span style="color:#179c80"><b>Proposal Distribution</b></span> genannt. Mithilfe der gezogen Samples kann dann versucht werden <span style="color:#db8642">analytisch nicht</span> (oder nur sehr aufwendig) lösbare Problem zu <span style="color:#db8642">lösen</span>. 
In einem ersten Experiment soll es nun darum gehen, eine sample-basierte Repräsentation einer Gauß-Verteilung zu berechnen.




<div class="alert alert-block alert-info" style="border-color:white">
    <b>Aufgabe</b>: Ziehen Sie <em>num_samples</em> einer konstanten Gauß Proposal Distribution mit $\mu=$ <em>self.mu</em> und $\sigma=$ <em>self.sigma</em>. Anschließend konvertieren Sie das Ergebnis zu einer <em>list</em> und fügen es <em>self.samples</em> hinzu. Ihren Code fügen Sie bitte in die folgende <em>do_step</em> Methode ein.
</div>

<div class="alert alert-block alert-warning" style="background-color:#f2decc
;border-color:white">
    <b>Hilfreiche Funktionen</b>: 
    <ul>
        <li> <em>np.random.normal($\mu$, $\sigma$, size)</em> </li>
    <li> <em>.tolist()</em></li>
    </ul>
</div>

<div class="alert alert-block alert-danger" style="border-color:white">
    <b>Wichtig!</b>: Bei Änderungen der <em>do_step</em> Methode, führen Sie die <b>drei</b> nachfolgenden Zellen erneut aus. 
</div>

In [1]:
# Definition der Grundklasse und GUI Elemente [keine Änderung notwendig]
# imports
import ipywidgets as widgets
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output
import ipywidgets
from ipywidgets import HBox, VBox, RadioButtons, Layout
import matplotlib.pyplot as plt
import seaborn as sbn
from scipy.stats import beta, binom, norm
import numpy as np
from ipywidgets import *
from helper.debounce import debounce
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output, display

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
import math
import jdc
mu_label = widgets.Label(value="Mittelwert $\mu$:")
mu_var = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1)
sigma_label = widgets.Label(value="Standardabweichung $\sigma$:")
sigma_var = widgets.FloatSlider(value=1, min=0, max=1, step=0.1)
samples_label = widgets.Label(value="Anzahl Samples:")
samples_var = widgets.IntSlider(value=10, min=1, max=1000, step=20)
btn = widgets.Button(description="Start")
btn2 = widgets.Button(description="Reset")
dr_label = widgets.Label(value="Samples pro Schritt", width='10px')
dr = widgets.Dropdown(
    options=['1', '10', '100'],
    value='1',
    disabled=False, layout=Layout(width="50px")
)
out = widgets.Output()

class MC_Sampler():
    def __init__(self, out):
        np.random.seed(42)
        self.out = out
        self.samples = []
        self.started = []
        self.mu, self.sigma, self.fig, self.ax = None, None, None, None
        with self.out:
            self.init_plot("?", "?", 0)
            plt.show(self.fig)
    def init_plot(self, mu, sigma, num_samples):
        sbn.set()
        self.fig, self.ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
        self.ax[0].set_ylabel(r"$\theta$")
        self.ax[1].set_xlabel(r"$\theta$")
        self.ax[0].set_title(fr"Aktuelle Proposal Distribution: $\mu=${mu}, $\sigma$={sigma}")
        self.ax[1].set_title(fr"Histogram über {num_samples} Sample")
        plt.subplots_adjust(wspace=0.4)
    
    def reset(self, btnn):
        self.started = False
        mu_var.disabled = False
        sigma_var.disabled = False
        btn.description="Start"
        self.samples = []
        self.mu, self.sigma = None, None
        with self.out:
            clear_output(wait=True)
            self.init_plot("?", "?", 0)
            plt.show(self.fig) 
    
    def init_step(self):
        if self.mu is None:
            self.mu = mu_var.value
            self.sigma = sigma_var.value

        num_samples = dr.value
        btn.description="Next"

        if not self.started:
            mu_var.disabled = True
            sigma_var.disabled = True
            self.started = False
        return int(num_samples)
    
    def plot(self):
        if len(self.samples)>0:     
            self.ax[0].plot(np.arange(len(self.samples)), self.samples)
            count, bins, ignored = self.ax[1].hist(self.samples, 30, density=False)
            self.ax[0].plot(len(self.samples)-1, self.samples[-1], linestyle="none", marker=".", markersize=5)
        else:
            print("Noch keine Samples erzeugt.")
        clear_output(wait=True)
        plt.show(self.fig) # [keine Änderung notwendig]

In [2]:
%%add_to MC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()

    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples)+num_samples)

        #self.samples += ...

        self.plot()        

In [3]:
# Starten der GUI [keine Änderung notwendig] 
MC_Sampler.do_step = do_step
mc_sampler = MC_Sampler(out=out)
btn.on_click(mc_sampler.do_step)
btn2.on_click(mc_sampler.reset)
widgets.HBox([widgets.VBox([mu_label, mu_var, sigma_label, sigma_var, widgets.HBox([widgets.VBox([btn, btn2]), widgets.VBox([dr_label, dr])])]), out]) # [keine Änderung notwendig]

###### <div class="alert alert-block alert-success" style="background-color:#b6dad2;color:#179c80;border-color:white"> <b>Lösung</b> <small style="color:#179c80">(Klicken auf den Pfeil um die Lösung aufzuklappen)</small></span></div>

In [8]:
%%add_to MC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()

    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples)+num_samples)

        self.samples  += np.random.normal(self.mu, self.sigma, num_samples).tolist()

        self.plot()  

#### 
<img src="./imgs/div.png" width="100%" /> 

 
## 1.3 Markov Chain Monte Carlo Experiment: Random Walk
Nun wissen wir, was ein Monte Carlo Verfahren ist - bleibt noch der Begriff <span style="color:#179c80"><b>Markov Chain</b> </span> offen.

Eine <span style="color:#179c80"><b>Markov Chain</b> </span> ist ein stochastischer Prozess, welcher eine <span style="color:#db8642">Folge</span> möglicher Ereignisse beschreibt. Charakteristisch hierbei ist, dass  die Wahrscheinlichkeit <span style="color:#db8642">jedes</span> Ereignisses 
vom <span style="color:#db8642">vorangegangenen</span> Zustand <span style="color:#179c80"><b>abhängig</b> </span> ist - sozusagen wie ein <span style="color:#db8642">kurzes Gedächtnis</span>. Alle davor liegenden Werte besitzen <span style="color:#db8642">keinen</span> Einfluss. 

<br>
Wie können wir nun das vorherige Monte Carlo Experiment zu einer <span style="color:#179c80"><b>Markov Chain</b> </span> erweitern? Die Idee ist recht simpel. Anstatt in jeder Iteration zufällig ein Sample von der gleichen Proposal Distribution zu ziehen, gestalten wir das Ziehen von Samples <span style="color:#db8642">abhängig</span> vom vorangegangenen Sample, indem wir die Proposal Distribution in jeder Iteration anpassen.


<div class="alert alert-block alert-info"  style="border-color:white">
    <b>Aufgabe</b>: Ziehen Sie insgesamt <em>num_samples</em> von einer Normalverteilung, bei welcher  $\mu$ dem zuvor gewählten Sample entspricht, also <em>self.mu = self.samples[-1]</em>. Das Verfahren ist dementsprechend iterativ zu implementieren. $\sigma$ bleibt konstant. Anschließend konvertieren Sie das Ergebnis zu einer <em>list</em> und fügen es <em>self.samples</em> hinzu. 
</div>

<div class="alert alert-block alert-warning" style="background-color:#f2decc
;border-color:white">
    <b>Wichtige Klassenattribute</b>: 
    <ul>
        <li> <em>num_samples</em>: enthält die Anzahl der Samples, welche im aktuellen Schritt gezogen werden sollen </li>
    <li> <em>self.samples</em>: fortlaufende Liste mit den gezogenen Samples </li>
        <li> <em>self.mu, self.sigma</em>: Momente der Proposal Distribution </li>
    </ul>
</div>
<div class="alert alert-block alert-danger" style="border-color:white">
    <b>Wichtig!</b>: Bei Änderungen der <em>do_step</em> Methode, führen Sie die <b>drei</b> nachfolgenden Zellen erneut aus. 
</div>

In [4]:
# Definition der Grundklasse und GUI Elemente [keine Änderung notwendig]

mu_label = widgets.Label(value="Mittelwert $\mu$:")
mu_var = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1)
sigma_label = widgets.Label(value="Standardabweichung $\sigma$:")
sigma_var = widgets.FloatSlider(value=1, min=0, max=1, step=0.1)
samples_label = widgets.Label(value="Anzahl Samples:")
samples_var = widgets.IntSlider(value=10, min=1, max=1000, step=20)
btn = widgets.Button(description="Start")
btn2 = widgets.Button(description="Reset")
dr_label = widgets.Label(value="Samples pro Schritt", width='10px')
dr = widgets.Dropdown(
    options=['1', '10', '100'],
    value='1',
    disabled=False, layout=Layout(width="50px")
)
out = widgets.Output()

class MCMC_Sampler():
    def __init__(self, out):
        np.random.seed(42)
        self.out = out
        self.samples = []
        self.started = []
        self.mu, self.sigma, self.fig, self.ax = None, None, None, None
        with self.out:
            self.init_plot("?", "?", 0)
            plt.show(self.fig)
    def init_plot(self, mu, sigma, num_samples):
        sbn.set()
        self.fig, self.ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
        self.ax[0].set_ylabel(r"$\theta$")
        self.ax[1].set_xlabel(r"$\theta$")
        self.ax[0].set_title(fr"Aktuelle Proposal Distribution: $\mu=${mu}, $\sigma$={sigma}")
        self.ax[1].set_title(fr"Histogram über {num_samples} Sample")
        plt.subplots_adjust(wspace=0.4)
    
    def reset(self, btnn):
        self.started = False
        mu_var.disabled = False
        sigma_var.disabled = False
        btn.description="Start"
        self.samples = []
        self.mu, self.sigma = None, None
        with self.out:
            clear_output(wait=True)
            self.init_plot("?", "?", 0)
            plt.show(self.fig) 
    
    def init_step(self):
        if self.mu is None:
            self.mu = mu_var.value
            self.sigma = sigma_var.value

        num_samples = dr.value
        btn.description="Next"

        if not self.started:
            mu_var.disabled = True
            sigma_var.disabled = True
            self.started = False
        return int(num_samples)
    
    def plot(self):
        if len(self.samples)>0:     
            self.ax[0].plot(np.arange(len(self.samples)), self.samples)
            count, bins, ignored = self.ax[1].hist(self.samples, 30, density=False)
            self.ax[0].plot(len(self.samples)-1, self.samples[-1], linestyle="none", marker=".", markersize=5)
            self.ax[0].set_title(fr"Aktuelle Proposal Distribution: $\mu=${'%.3f'%self.mu}, $\sigma$={self.sigma}")
        else:
            print("Noch keine Samples erzeugt.")
        clear_output(wait=True)
        plt.show(self.fig) 

In [5]:
%%add_to MCMC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()

    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples)+num_samples)
        
        #for i in range(num_samples):
            #self.samples  += ...
            #self.mu = ...
        
        self.plot()        

In [6]:
# Starten der GUI [keine Änderung notwendig]
MCMC_Sampler.do_step = do_step
mcmc_sampler = MCMC_Sampler(out=out)
btn.on_click(mcmc_sampler.do_step)
btn2.on_click(mcmc_sampler.reset)
widgets.HBox([widgets.VBox([mu_label, mu_var, sigma_label, sigma_var, widgets.HBox([widgets.VBox([btn, btn2]), widgets.VBox([dr_label, dr])])]), out]) 

###### <div class="alert alert-block alert-success" style="background-color:#b6dad2;color:#179c80;border-color:white"> <b>Lösung</b> <small style="color:#179c80">(Klicken auf den Pfeil um die Lösung aufzuklappen)</small></span></div>

In [ ]:
%%add_to MCMC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()

    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples)+num_samples)
        
        for i in range(int(num_samples)):
            self.samples  += np.random.normal(self.mu, self.sigma, 1).tolist()
            self.mu = self.samples[-1]
        
        self.plot()  

#### 
<img src="./imgs/div.png" width="100%" /> 

 
## 1.4: Metropolis-Hastings Algorithmus

Nach erfolgreichem Umsetzen des vorherigen Experiments fragen Sie sich vielleicht schon, wie man dieses Verfahren für die <span style="color:#179c80"><b>Posterior</b> </span> Berechnung verwenden kann - denn das <span style="color:#db8642">Akzeptieren</span> eines jeden Samples in jeder Iteration scheint nicht wirklich sinnvoll. Und genau hier besteht der Unterschied zum Metropolis-Hastings Algorithmus!

<br>
Um mit dem <span style="color:#179c80"><b>Metropolis-Hastings</b> </span> Algorithmus eine sample-basierte Posterior Verteilung zu berechnen, müssen Samples mit einer <span style="color:#db8642">Häufigkeit</span>, die im <span style="color:#db8642">Verhältnis</span> zur Posterior-Verteilung steht, gezogen werden. D.h. für ein $\theta$ mit einer hohen Posterior-Wahrscheinlichkeit wollen wir auch viele Samples nahe diesem Wert in unserem Sample-Pool. Doch wie ziehen wir Samples im richtigen Verhältnis von einer (normalerweise) <span style="color:#db8642">nicht</span> analytisch berechenbaren Posterior? Der Schlüssel liegt hierbei im Wort <span style="color:#179c80"><b>Verhältnis</b> </span>. Schauen wir uns einmal die grobe Funktionsweise des Algorithmus an:

> 1. Ziehe ein Sample $\theta_{\text{neu}}$ von einer Proposal Distribution, z.B. eine Gaußverteilung mit $\mu=\theta_{\text{alt}}$
> 2. Berechne das Verhältnis r = $\frac{\text{posterior}(\theta_{\text{neu}})}{\text{posterior}(\theta_{\text{alt}})}$
> 3. Berechnung einer Aktzeptanz-Warscheinlichkeit von $\theta_{\text{neu}}$ basierend auf $r$.
> 4. Abhängig von $r$ wird $\theta_{\text{neu}}$ entweder verworfen oder akzeptiert. Wird $\theta_{\text{neu}}$ verworfen, kehre zu 1. zurück. Falls $\theta_{\text{neu}}$ dem Sample-Pool hinzugefügt wird, setze $\theta_{\text{alt}}=\theta_{\text{neu}}$ und kehre zu 1. zurück.

Was also den Metropolis-Hastings Algorithmus <span style="color:#db8642">berechenbar</span> macht, ist die Verwendung des <span style="color:#179c80"><b>Verhältnisses</b> </span> $r = \frac{\text{posterior}(\theta_{\text{neu}})}{\text{posterior}(\theta_{\text{alt}})}$, da hierfür der <span style="color:#db8642">konstante</span>, hoch-dimensionale <span style="color:#db8642">Evidenz</span>-Term in der Posterior Berechnung vernachlässigt werden kann.



<div class="alert alert-block alert-info"  style="border-color:white">
    <b>Aufgabe</b>: Setzen Sie den MH-Algorithmus um. Hierfür implementieren Sie folgende Schritte in der nachfolgenden <em>do_step</em> Methode:
    <ol>
        <li>Iterieren Sie über <em>num_samples</em>.</li>
        <li>Falls noch keine Samples in <em>self.samples</em> enthalten sind, ziehen Sie ein Sample von $\mathcal{N}($ <em>self.mu</em>, <em>self.sigma</em> $)$ und fügen Sie es <em>self.samples</em> hinzu.</li>
        <li>Falls <em>self.samples</em> bereits Elemente enthält:
        <ol>
            <li> Ziehen Sie ein Sample <em>candidate</em> von  $\mathcal{N}($ <em>self.mu</em>, <em>self.sigma</em> $)$.</li>
            <li> Weisen Sie dem letzten, dem Pool hinzugefügten Sample, den Namen  <em>last_sample</em> zu.</li> 
            <li> Berechnen Sie $r$, indem Sie den Posterior-Wert für <em>candidate</em> und <em>last_sample</em> mithilfe des Prior-Werts <em>beta.pdf(?, 1, 1)</em> und des Likelihood-Werts <em>binom.pmf(n=10, k=4, p=?)</em> berechnen.</li>
            <li> Erstellen Sie eine Variable <em>acceptance_prob</em> mit dem Wert $\text{minimum}(r, 1)$.</li>
            <li> Ziehen Sie ein Sample <em>u</em> einer Uniform Verteilung zwischen $0$ und $1$.</li>
            <li> Wenn <em>u</em> < <em>acceptance_prob</em>, dann akzeptieren wir <em>candidate</em>, d.h. Sie fügen ihn <em>self.samples</em> hinzu und setzen den Mittelwert der Proposal Verteilung auf den Wert <em>candidate</em>. Falls <em>u</em>>=<em>acceptance_prob</em>, endet die Iteration.</li>
         </ol>
        </li>
    </ol>
</div>



<div class="alert alert-block alert-warning" style="background-color:#f2decc
;border-color:white">
    <b>Hilfreiche Funktionen</b>: 
    <ul>
        <li> <em>np.random.normal(...)[0]</em> </li>
    <li> <em>.tolist()</em></li>
        <li> <em>self.samples[-1]</em></li>
        <li> <em>beta.pdf(?, 1, 1)</em></li>
        <li> <em>binom.pmf(n=10, k=4, p=?)</em></li>
        <li> <em>min(a, b)</em></li>
        <li> <em>np.random.uniform(...)[0]</em></li>
    </ul>
</div>

<div class="alert alert-block alert-danger" style="border-color:white">
    <b>Wichtig!</b>: Bei Änderungen der <em>do_step</em> Methode, führen Sie die <b>drei</b> nachfolgenden Zellen erneut aus. 
</div>

In [7]:
# Definition der Grundklasse und GUI Elemente [keine Änderung notwendig]
np.random.seed(42)
mu_label = widgets.Label(value="$\mu$ initial:")
mu_var = widgets.FloatSlider(value=0.3, min=0.2, max=0.6, step=0.1)
sigma_label = widgets.Label(value="$\sigma$ initial:")
sigma_var = widgets.FloatSlider(value=1, min=0.1, max=0.5, step=0.1)
samples_label = widgets.Label(value="Anzahl Samples:")
samples_var = widgets.IntSlider(value=10, min=1, max=1000, step=20)
btn = widgets.Button(description="Start")
btn2 = widgets.Button(description="Reset")
dr_label = widgets.Label(value="Samples pro Schritt", width='10px')
dr = widgets.Dropdown(
    options=['1', '10', '100', '1000'],
    value='1',
    disabled=False, layout=Layout(width="50px")
)
info_label = widgets.Label(value="Iterationen: 0 | Verworfen: 0 | Akzeptiert: 0")
out = widgets.Output()

class MCMC_Sampler():
    def __init__(self, out):
        np.random.seed(42)
        self.out = out
        self.samples = []
        self.iter = 0
        self.all_samples = []
        self.started = []
        self.mu, self.sigma, self.fig, self.ax = None, None, None, None
        with self.out:
            self.init_plot("?", "?", 0)
            plt.show(self.fig)
    def init_plot(self, mu, sigma, num_samples):
        sbn.set()
        self.fig, self.ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
        self.ax[0].set_ylabel(r"$\theta$")
        self.ax[1].set_xlabel(r"$\theta$")
        self.ax[0].set_title(fr"Aktuelle Proposal Distribution: $\mu=$?, $\sigma$=?")
        self.ax[1].set_title(fr"Histogram über {num_samples} Samples")
        # plot posterior
        xx = np.linspace(0, 1, 1000)
        self.ax[1].plot(xx, beta.pdf(xx, 5, 7), label="Beta(5,7)")
        self.ax[1].legend()
        self.ax[1].set_xlim([-0.1, 1.1])
        plt.subplots_adjust(wspace=0.4)
        return 
    
    def reset(self, btnn):
        self.started = False
        mu_var.disabled = False
        sigma_var.disabled = False
        btn.description="Start"
        self.mu, self.sigma = None, None
        self.samples = []
        self.all_samples = []
        info_label.value = "Iterationen: 0 | Verworfen: 0 | Akzeptiert: 0"
        self.iter = 0
        with self.out:
            clear_output(wait=True)
            self.init_plot("?", "?", 0)
            plt.show(self.fig)  
    
    def init_step(self):
        if self.mu is None:
            self.mu = mu_var.value
            self.sigma = sigma_var.value

        num_samples = dr.value
        btn.description="Next"

        if not self.started:
            mu_var.disabled = True
            sigma_var.disabled = True
            self.started = False
        return int(num_samples)
    
    def plot(self):

        if len(self.samples)>0:     
            #self.ax[0].set_title(fr"new= -,old= - ")
            self.ax[1].set_title(fr"Histogram über {len(self.samples)} Sample")
            self.ax[0].set_title(fr"Aktuelle Proposal Distribution: $\mu=${'%.3f'%self.mu}, $\sigma$={self.sigma}")

            self.ax[0].plot(np.arange(len(self.all_samples)), self.all_samples)
            count, bins, ignored = self.ax[1].hist(self.samples, 30, density=True, stacked=True)
            info_label.value = f"Iterationen: {self.iter} | Verworfen: {self.iter-len(self.samples)} | Akzeptiert: {len(self.samples)}"
            # finalize layout
            #ax[0].plot([10 for _ in range(len(samples))], samples, linestyle="none", marker=".", markersize=5)
            self.ax[0].plot(len(self.all_samples)-1, self.all_samples[-1], linestyle="none", marker=".", markersize=5)
        else:
            print("Noch keine Samples erzeugt.")
        clear_output(wait=True)
        plt.show(self.fig) 

In [8]:
%%add_to MCMC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()

    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples))
        self.iter += int(num_samples)

        for _ in range(int(num_samples)):
            if len(self.samples) == 0:
                # candidate  = ...                  
                # self.samples += ...
                # self.all_samples += [candidate] # Kommentieren Sie diese Zeile ebenfalls aus
                pass
            else:
                #candidate = ...
                #last_sample = ...

                #post_new = ...
                #post_old = ...

                #ratio = ...
                #acceptance_prob = ...
                #u = ...

                #if u < acceptance_prob:
                    #self.samples +=
                #self.all_samples += [candidate] # Kommentieren Sie diese Zeile ebenfalls aus
                pass
                
            #self.mu = ...
            
        self.plot()

In [9]:
# Starten der GUI [keine Änderung notwendig]
MCMC_Sampler.do_step = do_step
mcmc_sampler = MCMC_Sampler(out=out)
btn.on_click(mcmc_sampler.do_step)
btn2.on_click(mcmc_sampler.reset)
widgets.HBox([widgets.VBox([mu_label, mu_var, sigma_label, sigma_var, widgets.HBox([widgets.VBox([btn, btn2]), widgets.VBox([dr_label, dr])]), info_label]), out])

###### <div class="alert alert-block alert-success" style="background-color:#b6dad2;color:#179c80;border-color:white"> <b>Lösung</b> <small style="color:#179c80">(Klicken auf den Pfeil um die Lösung aufzuklappen)</small></span></div>

In [ ]:
%%add_to MCMC_Sampler
def do_step(self, btn):       
    num_samples = self.init_step()


    with self.out:
        self.init_plot(self.mu, self.sigma, len(self.samples))
        self.iter += int(num_samples)

        for _ in range(int(num_samples)):
            if len(self.samples) == 0:
                    candidate  = np.random.normal(self.mu, self.sigma, 1)[0]
                    #post_new = beta.pdf(candidate, 1, 1)*binom.pmf(n=10, k=4, p=candidate) 
                    #while np.isnan(post_new):
                        #candidate  = np.random.normal(self.mu, self.sigma, 1)[0]
                        #post_new = beta.pdf(candidate, 1, 1)*binom.pmf(n=10, k=4, p=candidate)
                    self.samples += [candidate]
                    self.all_samples += [candidate]
            else:
                candidate = np.random.normal(self.mu, self.sigma, 1)[0]
                last_sample = self.samples[-1]

                post_new = beta.pdf(candidate, 1, 1)*binom.pmf(n=10, k=4, p=candidate) 
                post_old = beta.pdf(last_sample, 1, 1)*binom.pmf(n=10, k=4, p=last_sample)

                ratio = post_new / post_old
                acceptance_prob = min(ratio, 1)
                u = np.random.uniform(low=0, high=1, size=1)[0]

                if u < acceptance_prob:
                    self.samples += [candidate]
                self.all_samples += [candidate]


            self.mu = self.samples[-1]
            
        self.plot()

#### 
<img src="./imgs/div.png" width="100%" /> 

  
## 1.5 MCMC und Neuronale Netze
Selbes Prinzip kann nun für Neuronale Netze angewendet werden. Hierfür werden die <span style="color:#179c80"><b>Gewichte</b> </span> als Verteilungen betrachtet, von welchen über <span style="color:#179c80"><b>MCMC</b> </span> eine <span style="color:#db8642">sample-basierte</span> Posterior Repräsentation abgeleitet wird. Mithilfe dieser Verteilungen können Unsicherheitsangaben über Gewichte getroffen werden (z.B. mittels Varianz in den Samples).

Für eine Modellvorhersage wird jeweils pro Gewichtsverteilung ein Sample gezogen und mit dieser Punktschätzung eine Vorhersage berechnet.  Dies geschiet <span style="color:#db8642">T</span> mal, wodurch man über die Ausgaben ebenfalls eine <span style="color:#db8642">Verteilung</span>  bilden kann.
<img src="./imgs/mcmcc.png" width=100%>